# Setup command line client

The following is code to get the client and set it up.

First we jupyter bookmark the current dir

In [2]:
%bookmark root
import os
os.getcwd()

'/home/abr/Downloads/bitmag-release1.9-test'

In [3]:
#Some imports we will need later
import random
import string

Switch back to root and set up the client name

In [4]:
%cd -b root
%bookmark CLIENT bitrepository-client-1.9-RC1
%env CLIENT bitrepository-client-1.9-RC1
%bookmark -l

(bookmark:root) -> /home/abr/Downloads/bitmag-release1.9-test
/home/abr/Downloads/bitmag-release1.9-test
env: CLIENT=bitrepository-client-1.9-RC1
Current bookmarks:
CLIENT -> bitrepository-client-1.9-RC1
root   -> /home/abr/Downloads/bitmag-release1.9-test


Download the client and unzip it

In [5]:
%%bash
pwd
wget -Nq "https://sbforge.org/download/attachments/25395346/${CLIENT}.zip"
rm -rf ${CLIENT}
unzip -q ${CLIENT}.zip

/home/abr/Downloads/bitmag-release1.9-test


Change directory to the unzipped client

In [6]:
%cd -b CLIENT

(bookmark:CLIENT) -> bitrepository-client-1.9-RC1
/home/abr/Downloads/bitmag-release1.9-test/bitrepository-client-1.9-RC1


# Describes the tests needed to validate the PutFile functionality.

The commands presented are just examples of what the actual command to test with could look like.

## Basic put

### Put a file to all pillars

Run put command on a file which is not present in the collection.

In [7]:
%env TESTFILE1 logback.xml

env: TESTFILE1=logback.xml


Check this by running 

In [8]:
!bin/bitmag.sh get-file-ids -c integrationtest2 -i $TESTFILE1

Count: 	Size: 	FileID:
5 	641 	logback.xml


The response should be

    Count: 	Size: 	FileID:

Indicating that no pillar had the file.
If the file is found, delete it with the following command

In [10]:
%%bash
hash=$(cat conf/$TESTFILE1 | md5sum - | cut -d' ' -f1)
bin/bitmag.sh delete -c integrationtest2 -i $TESTFILE1 -C $hash -p "sbtape2"
bin/bitmag.sh delete -c integrationtest2 -i $TESTFILE1 -C $hash -p "reference2"
bin/bitmag.sh delete -c integrationtest2 -i $TESTFILE1 -C $hash -p "checksum2"
bin/bitmag.sh delete -c integrationtest2 -i $TESTFILE1 -C $hash -p "sbdisk1"
bin/bitmag.sh delete -c integrationtest2 -i $TESTFILE1 -C $hash -p "kbpillar2"

In [11]:
!bin/bitmag.sh put-file -c integrationtest2 -f conf/$TESTFILE1

Failure: f6834bdd: PUT_FILE for file logback.xml: FAILED: , Failed operation. Cause(s):
[f6834bdd: PUT_FILE for file logback.xml: COMPONENT_FAILED: ContributorID kbpillar2 ResponseCode: FILE_TRANSFER_FAILURE, File transfer for logback.xml to http://int-bitmag-01.kb.dk:80/dav/logback.xml]


Verify that it is now present at all pillars, and that it has the same checksum at all pillars

In [12]:
!bin/bitmag.sh get-file-ids -c integrationtest2 -i $TESTFILE1

Count: 	Size: 	FileID:
4 	641 	logback.xml


In [13]:
!bin/bitmag.sh get-checksums -c integrationtest2 -i $TESTFILE1

Count: 	Checksum: 	Pillars: 	FileID: 
4 	3228f1d3684c39ed782af5f9e6aee1b0 	All 	logback.xml


### Put a file with a different file-id to all pillars

Run put command on a file and use the file-id parameter (-i)

In [14]:
RANDOMNAME=''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(10))
%env TESTFILE2=$RANDOMNAME

env: TESTFILE2=EBX07MMSKU


In [15]:
!bin/bitmag.sh put-file -c integrationtest2 -f conf/$TESTFILE1 -i $TESTFILE2

Verify that it is now present at all pillars, and that it has the same checksum at all pillars    

In [16]:
!bin/bitmag.sh get-file-ids -c integrationtest2 -i $TESTFILE2

Count: 	Size: 	FileID:
5 	641 	EBX07MMSKU


In [17]:
!bin/bitmag.sh get-checksums -c integrationtest2 -i $TESTFILE2

Count: 	Checksum: 	Pillars: 	FileID: 
5 	3228f1d3684c39ed782af5f9e6aee1b0 	All 	EBX07MMSKU


### Put a file by using an URL

Use the URL and Checksum to put a file, already on the webdav server.

In [18]:
RANDOMNAME=''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(10))
%env TESTFILE3=$RANDOMNAME

%env URL http://sandkasse-01.kb.dk/dav/test.txt

env: TESTFILE3=81K86TRWDP
env: URL=http://sandkasse-01.kb.dk/dav/test.txt


In [19]:
%%bash
hash=$(curl -s $URL | md5sum - | cut -d' ' -f1)
echo "md5: $hash"
bin/bitmag.sh put-file -c integrationtest2 -u $URL -i $TESTFILE3 -C $hash

md5: 940a51b250e7aa82d8e8ea31217ff267


Verify that it is now present at al pillars, with the requested checksum

In [20]:
!bin/bitmag.sh get-file-ids -c integrationtest2 -i $TESTFILE3

Count: 	Size: 	FileID:
5 	27 	81K86TRWDP


In [21]:
!bin/bitmag.sh get-checksums -c integrationtest2 -i $TESTFILE3

Count: 	Checksum: 	Pillars: 	FileID: 
5 	940a51b250e7aa82d8e8ea31217ff267 	All 	81K86TRWDP


## Idempotent test

### Attempt to put a file with an fileID that is already present in the collection suppling the same checksum

The putFile request should succeed.

In [22]:
!bin/bitmag.sh put-file -c integrationtest2 -f conf/$TESTFILE1

## Put with returned checksums

### Put a file including a request of a salted checksum calculated on the pillar, which should be returned.

In [23]:
import hmac
import hashlib
import urllib.request

def getSaltedChecksum(url,salt,algorithm):
    saltBytes = bytes.fromhex(salt)
    digester = hmac.new(saltBytes,None,algorithm)
    with urllib.request.urlopen(url) as from_fh:
        while True:
            chunk = from_fh.read()
            if not chunk:
                break
            digester.update(chunk)
    return digester.hexdigest().lower()

saltedChecksum=getSaltedChecksum(url=os.environ.get("URL"),salt='abcd',algorithm=hashlib.sha1)
print(saltedChecksum)

5bb8cf7554e747f7ab8bd09a910b680bf889d959


Use the request-checksum-algorithm and request-checksum-salt arguments (-R and -S):

In [24]:
RANDOMNAME=''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(10))
%env TESTFILE4=$RANDOMNAME

env: TESTFILE4=VO78OYQOLG


Note, this will return the stored MD5 hashes, if the file is already put'et

In [25]:
%%bash
hash=$(curl -s $URL | md5sum - | cut -d' ' -f1)
echo "md5: $hash"
bin/bitmag.sh put-file -c integrationtest2 -u $URL -i $TESTFILE4 -C $hash -S 'abcd' -R HMAC_SHA1

md5: 940a51b250e7aa82d8e8ea31217ff267
PillarId 	 Checksum
kbpillar2 	 5bb8cf7554e747f7ab8bd09a910b680bf889d959
sbtape2 	 5bb8cf7554e747f7ab8bd09a910b680bf889d959
reference2 	 5bb8cf7554e747f7ab8bd09a910b680bf889d959
sbdisk1 	 5bb8cf7554e747f7ab8bd09a910b680bf889d959


Verify that the checksumpillar does not reply with a checksum.

In [26]:
!bin/bitmag.sh get-checksums -c integrationtest2 -i $TESTFILE4 -R HMAC_SHA1 -S 'abcd'

Count: 	Checksum: 	Pillars: 	FileID: 
4 	5bb8cf7554e747f7ab8bd09a910b680bf889d959 	All 	VO78OYQOLG


Verify that the checksumpillar and all the other pillars have the file

In [27]:
!bin/bitmag.sh get-file-ids -c integrationtest2 -i $TESTFILE4

Count: 	Size: 	FileID:
5 	27 	VO78OYQOLG


### Put a file including a request for checksum, but using both a salt and a non-salt algorithm.

Use the request-checksum-algorithm and request-checksum-salt arguments (-R and -S):

In [28]:
RANDOMNAME=''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(10))
%env TESTFILE5=$RANDOMNAME

env: TESTFILE5=BUDOXER44F


In [29]:
%%bash
hash=$(curl -s $URL | md5sum - | cut -d' ' -f1)
echo "md5: $hash"
bin/bitmag.sh put-file -c integrationtest2 -u $URL -i $TESTFILE5 -C $hash -R SHA1 -S 'abcd'

md5: 940a51b250e7aa82d8e8ea31217ff267
PillarId 	 Checksum
kbpillar2 	 5bb8cf7554e747f7ab8bd09a910b680bf889d959
sbtape2 	 5bb8cf7554e747f7ab8bd09a910b680bf889d959
reference2 	 5bb8cf7554e747f7ab8bd09a910b680bf889d959
sbdisk1 	 5bb8cf7554e747f7ab8bd09a910b680bf889d959


Verify that all the data pillars return the same checksum, and that it is identical to the previous test (where '-R HMAC_SHA1')

### Put a file including a request for checksum, but using a salt algorithm but not giving a salt.

Use the request-checksum-algorithm and request-checksum-salt arguments (-R and no -S):

In [30]:
%%bash
hash=$(curl -s $URL | md5sum - | cut -d' ' -f1)
echo "md5: $hash"
hash=$(curl -s $URL | sha1sum - | cut -d' ' -f1)
echo "sha1: $hash"
bin/bitmag.sh get-checksums -c integrationtest2 -i $TESTFILE5

md5: 940a51b250e7aa82d8e8ea31217ff267
sha1: 82b4b156a5db351e278e3a8e5d3e762deca45fdb
Count: 	Checksum: 	Pillars: 	FileID: 
5 	940a51b250e7aa82d8e8ea31217ff267 	All 	BUDOXER44F


Verify that the data pillars all deliver the same checksum, which must be different from the checksum in the previous two tests.

## Error scenarios

### Attempt to put a file with an fileID that is already present in the collection (ensure that the file has a different checksum from the already archived)

The putFile request should fail informing the user that a file with the given file ID already exists in the collection.

In [31]:
!bin/bitmag.sh put-file -c integrationtest2 -f conf/RepositorySettings.xml -i $TESTFILE1

Failure: 811e91d7: PUT_FILE for file logback.xml: FAILED: , Can not put file logback.xml, as an different file already exists on pillar kbpillar2


### Attempt to put a file which does not exist

Client should fail immediately

In [32]:
!bin/bitmag.sh put-file -c integrationtest2 -f ThisFileDoesNotExist

java.lang.IllegalArgumentException: The file 'ThisFileDoesNotExist' is invalid. It does not exists or it is a directory.
	at org.bitrepository.commandline.CommandLineClient.findTheFile(CommandLineClient.java:283)
	at org.bitrepository.commandline.CommandLineClient.getURLOrUploadFile(CommandLineClient.java:329)
	at org.bitrepository.commandline.PutFileCmd.putTheFile(PutFileCmd.java:159)
	at org.bitrepository.commandline.PutFileCmd.performOperation(PutFileCmd.java:86)
	at org.bitrepository.commandline.CommandLineClient.runCommand(CommandLineClient.java:68)
	at org.bitrepository.commandline.PutFileCmd.main(PutFileCmd.java:50)


### Attempt to put a file to a non-existing collection

Client should fail immediately

In [33]:
!bin/bitmag.sh put-file -c integrationtest3 -i $TESTFILE1

Invalid argument: integrationtest3 is not a valid collection.
The following collections are defined: [integrationtest1, integrationtest2, reklamefilm, dvd, avis, netarkiv]


### Attempt to put a file to a non-existing pillar

Client should fail immediately

In [34]:
!bin/bitmag.sh put-file -c integrationtest1 -i $TESTFILE1 -p non-existing-pillar

Invalid argument: non-existing-pillar is not a valid pillar for collection integrationtest1
The collection contains the following pillars: [sbtape2, reference2, checksum2, sbdisk1, kbpillar2]
